# Se liga na música

# Conteúdo <a id='back'></a>

* [Introdução](#intro)
* [Etapa 1. Visão geral dos dados](#data_review)
    * [Conclusões](#data_review_conclusions)
* [Etapa 2. Pré-processamento de dados](#data_preprocessing)
    * [2.1 Estilo do cabeçalho](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusões](#data_preprocessing_conclusions)
* [Etapa 3. Teste das hipóteses](#hypotheses)
    * [3.1 Hipótese 1: atividade dos usuários nas duas cidades](#activity)
    * [3.2 Hipótese 2: preferências musicais na segunda e sexta-feira](#week)
    * [3.3 Hipótese 3: preferências de gênero em Springfield e Shelbyville](#genre)
* [Conclusões](#end)

## Introdução <a id='intro'></a>
Neste projeto irei comparar as preferências musicais dos habitantes de Springfild e Shelbyville. Estudarei os dados de um serviço de streaming de música online para testar as hipóteses apresentadas abaixo e comparar o comportamento dos usuários para essas duas cidades.

### Objetivo:
Testar as três hipóteses:
1. A atividade do usuário varia dependendo do dia da semana e da cidade.
2. Nas segundas-feiras de manhã, os habitantes de Springfield e Shelbyville escutam diferentes gêneros. Isso também é verdadeiro para noites de sexta-feira.
3. Ouvintes de Springfield e Shelbyville têm preferências diferentes. Em Springfield, as pessoas preferem pop, enquanto Shelbyville tem mais fãs de rap.

### Etapas
Os dados sobre o comportamento dos usuários estão armazenados no arquivo `/datasets/music_project_en.csv`. Não há informação sobre a qualidade dos dados, então eu os examinei antes de testar a hipótese.

Primeiro, avaliei a qualidade dos dados e vi se seus problemas eram significativos. Depois, durante o pré-processamento dos dados, tentei tratar dos problemas mais críticos.

O meu projeto consiste em três etapas:
 1. Visão geral dos dados
 2. Pré-processamento de dados
 3. Teste das hipóteses


[Back to Contents](#back)

## Etapa 1. Visão geral dos dados <a id='data_review'></a>

In [2]:
# importando pandas
import pandas as pd

Leia o arquivo `music_project_en.csv` da pasta `/datasets/` e armazene-o na variável `df`:

In [3]:
# lendo o arquivo e armazenando-o em df
df = pd.read_csv('/datasets/music_project_en.csv')

Impressão das primeiras 10 linhas da tabela:

In [5]:
# obtendo as primeiras 10 linhas da tabela df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


Informação geral sobre a tabela usando um comando:

In [6]:
# obter informação geral sobre os dados em df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Aqui estão as minhas observações sobre a tabela. Ela contém sete colunas. Elas armazenam o mesmo tipo de dados: `object`.

De acordo com a documentação:
- `'userID'` — identificação do usuário
- `'Track'` — título da música
- `'artist'` — nome do artista
- `'genre'` — gênero da música
- `'City'` — cidade do usuário
- `'time'` — tempo exato que a música foi tocada
- `'Day'` — dia da semana

Posso ver três problemas de estilo nos cabeçalhos da tabela:
1. Alguns cabeçalhos são escritos em letras maiúsculas, outros estão em minúsculas.
2. Alguns cabeçalhos contêm espaços.
3. 'userID' é uma coluna que deveria ser escrita utilizando "snake_case", pois é um nome com duas palavras: user e id.




### Minhas observações <a id='data_review_conclusions'></a>

Temos dados do tipo 'object' nas linhas, e as colunas são objetos do tipo Series que descrevem os atributos da atividade de um usuário, representada por uma linha. Isto é, cada música tocada possui informações associadas, e cada informação é representada por um valor registrado em uma coluna.

Esses dados são suficientes para responder às nossas três hipóteses, uma vez que as colunas disponibilizadas possibilitam realizar as manipulações necessárias para obter as informações. Veja:

1. 1ª Hipótese: são necessárias informações de dia da semana, cidade e usuário, representadas pelas colunas 'Day', '  City' e '  userID', respectivamente.
2. 2ª Hipótese: são necessárias informações de dia da semana, cidade, genêro e horário, representadas pelas colunas 'Day', '  City','genre' e 'time', respectivamente.
3. 3ª Hipótese: são necessárias informações de dia da semana, cidade, genêro e horário, representadas pelas colunas 'Day', '  City','genre' e 'time', respectivamente.

Por fim, Há valores ausentes em 3 colunas ('Track', 'artist' e 'genre'), 3826 linhas duplicadas e a coluna "time" deveria possuir dados do tipo "timestamp".

In [7]:
## Checagem de valores ausentes
df.isnull().sum()

  userID       0
Track       1343
artist      7567
genre       1198
  City         0
time           0
Day            0
dtype: int64

In [8]:
## Checagem de valores duplicados
df.duplicated().sum()

3826

[Voltar ao Índice](#back)

## Etapa 2. Pré-processamento de dados <a id='data_preprocessing'></a>

O objetivo aqui é preparar os dados para a análise.
O primeiro passo é resolver todos os problemas com o cabeçalho. E então podemos passar para os valores ausentes e duplicados. Vamos começar.


### Estilo do cabeçalho <a id='header_style'></a>

In [9]:
# a lista de cabeçalhos para a tabela df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [10]:
# Percorrendo os cabeçalhos e convertendo tudo em minúsculos
lower_columns = []

for col in df.columns:
    lower_columns.append(col.lower())
    
df.columns = lower_columns

In [11]:
# Percorrendo os cabeçalhos e removendo os espaços
stripped_columns = []

for col in df.columns:
    stripped_columns.append(col.strip())
    
df.columns = stripped_columns

In [12]:
# Renomeando o cabeçalho "userid"
df.rename(columns = {'userid': 'user_id'}, inplace = True)

In [13]:
# verificando o resultado: a lista de cabeçalhos
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

[Voltar ao Índice](#back)

### Valores ausentes <a id='missing_values'></a>

In [14]:
# calculando o número de valores ausentes
print(df.isnull().sum())
print(df.isna().sum())

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64
user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64


Nem todos os valores ausentes afetam a pesquisa. Por exemplo, os valores ausentes nas colunas `track` e `artist` não são críticos. Portanto, posso simplesmente substituí-los por valores padrão, como, por exemplo, a string `'unknown'`.

Mas valores ausentes em `'genre'` podem afetar a comparação de preferências musicais em Springfield e Shelbyville. Na vida real, seria útil descobrir as razões pelas quais os dados estão ausentes e tentar corrigi-los. Mas nós não temos essa possibilidade neste projeto. Então, eu apenas:
* Preenchi esses valores ausentes com um valor padrão
* Avaliei em que medida os valores ausentes podem afetar a sua análise

Substituição dos valores ausentes em `'track'`, `'artist'`, e `'genre'` pela string `'unknown'`. Para fazer isso, criei a lista `columns_to_replace`, a percorri com um ciclo `for` e substitua os valores ausentes em cada coluna:

In [15]:
# percorrendo os cabeçalhos e substituindo valores ausentes por 'unknown'
columns_to_replace = ['track', 'artist', 'genre']

for col in columns_to_replace:
    df[col].fillna('unknown', inplace=True)

In [16]:
# contando os valores ausentes
df.isnull().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

[Voltar ao Índice](#back)

### Valores duplicados <a id='duplicates'></a>

In [17]:
# contando duplicados explícitos
df.duplicated().sum()

3826

Chame o método `pandas` para se livrar dos duplicados explícitos:

In [18]:
# removendo duplicados explícitos
df.drop_duplicates(inplace=True)

Conte os duplicados explícitos mais uma vez para se certificar de que você removeu todos eles:

In [19]:
# verificando duplicados
df.duplicated().sum()

0

Agora queremos nos livrar dos duplicados implícitos na coluna `genre`. Por exemplo, o nome de um gênero pode ser escrito de maneiras diferentes. Alguns erros afetarão também o resultado.

Para fazer isso, vamos começar imprimindo uma lista de nomes de gênero unívocos, ordenados em ordem alfabética: Para fazer isso:
* Obtive a coluna pretendida do DataFrame
* Chamei o método que retornará todos os valores unívocos da coluna
* Apliquei um método de ordenação ao resultado


In [20]:
# visualizando nomes unívocos de gêneros
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

Encontei duplicados implícitos do gênero `hiphop`. Esses podem ser nomes escritos incorretamente, ou nomes alternativos para o mesmo gênero.

Você verá os seguintes duplicados implícitos:
* `hip`
* `hop`
* `hip-hop`

Para se livrar deles, declarei a função `replace_wrong_genres()` com dois parâmetros:
* `wrong_genres=` — a lista dos duplicados
* `correct_genre=` — a string com o valor correto

A função corrige os nomes na coluna `'genre'` da tabela `df`, isto é, substitui cada valor da lista `wrong_genres` pelo valor de `correct_genre`. Usei um ciclo `'for'` para percorrer a lista de gêneros errados e substituí-los pelo gênero correto na lista principal.

In [21]:
# função para substituir duplicados implícitos
def replace_wrong_genres(wrong_genres, correct_genre):
    for genre in wrong_genres:
        df['genre'] = df['genre'].replace(genre, correct_genre)
    return

In [22]:
# removendo duplicados implícitos
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop')

In [23]:
# verificando a existência de valores duplicados
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

[Voltar ao Índice](#back)

### Minhas observações <a id='data_preprocessing_conclusions'></a>

Duplicados podem ser tanto do tipo explícito(quando toda a linha se repete) ou pode ser do tipo implicito, quando há uma linha que diferencia-se apenas por alguma diferença de escrita, ortografia ou tipo de dado. No primeiro caso, eliminamos as linhas duplicadas ao utilizar o método drop_duplicates(), enquanto que na segunda situação torna-se necessário realizar uma inspeção manual, procurando e renomeando valores unívocos diferentes que expressam a mesma ideia, caso de "hiphop", "hip", "hop" e "hip-hop", que representam a mesma categoria "hiphop".

[Voltar ao Índice](#back)

## Etapa 3. Testando hipóteses <a id='hypotheses'></a>

### Hipótese 1: comparando o comportamento dos usuários nas duas cidades <a id='activity'></a>

A primeira hipótese afirma que existem diferenças no consumo de música pelos usuários em Springfield e os em Shelbyville. Para testar a hipótese, usei os dados dos três dias da semana: segunda-feira (Monday), quarta-feira (Wednesday) e sexta-feira (Friday).

* Agrupei os usuários por cidade.
* Comparei o número de músicas tocadas por cada grupo na segunda, quarta e sexta.


In [42]:
df_by_city = df.groupby('city')
# Contando as músicas tocadas em cada cidade
print(df_by_city['track'].count())

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64


Observa-se que Springfield ouve cerca de 2,3x mais música do que Shelbyville.

Em seguida agrupei os dados por dia da semana e encontre a quantidade de músicas tocadas na segunda, quarta e sexta-feira.


In [ ]:
# Calculando as músicas tocadas em cada dia da semana
df.groupby('day')['track'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

 Observa-se um aumento no consumo no início da semana(segunda-feira) e no final(Sexta-feira), com o número de músicas tocadas superando 21 mil. Em contrapartida, a quarta-feira é o dia com menos visualizações(18059).

In [ ]:
# <criando a função number_tracks()>
# Vamos declarar a função com dois parâmetros: day=, city=.
# Deixei a variável track_list armazenar as linha df onde
# o valor na coluna 'day' é igual ao parâmetro day= e ao mesmo tempo,
# o valor na coluna 'city' é igual ao parâmetro city= (aplique filtragem consecutiva
# com indexação lógica).
# Deixei a variável track_list_count armazenar o número de valores da coluna 'user_id' em track_list
# (igual ao número de linhas em track_list depois de filtrar duas vezes).
# Deixei a função retornar um número: o valor de track_list_count.

# A função conta músicas tocadas por uma certa cidade e dia.
# Primeiro retorna as linhas com o dia pretendido da tabela,
# depois filtra as linhas com a cidade pretendida a partir do resultado,
# em seguida, encontra o número de músicas na tabela filtrada,
# então retorna aquele número.

def number_tracks(day, city):
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [45]:
# a quantidade de músicas tocadas em Springfield na segunda-feira
print(number_tracks('Monday', 'Springfield'))

15740


In [46]:
# a quantidade de músicas tocadas em Shelbyville na segunda-feira
print(number_tracks('Monday', 'Shelbyville'))

5614


In [47]:
# a quantidade de músicas tocadas em Springfield na quarta-feira
print(number_tracks('Wednesday', 'Springfield'))

11056


In [48]:
# a quantidade de músicas tocadas em Shelbyville na quarta-feira
print(number_tracks('Wednesday', 'Shelbyville'))

7003


In [49]:
# a quantidade de músicas tocadas em Springfield na sexta-feira
print(number_tracks('Friday', 'Springfield'))

15945


In [50]:
# a quantidade de músicas tocadas em Shelbyville na sexta-feira
print(number_tracks('Friday', 'Shelbyville'))

5895


Usei `pd.DataFrame` para criar uma tabela, onde
* Os cabeçalhos da tabela são: `['city', 'monday', 'wednesday', 'friday']`
* Os dados são o resultado que você recebe de `number_tracks()`

In [51]:
# tabela com resultados
columns = ['city', 'monday', 'wednesday', 'friday']
days = ['Monday', 'Wednesday', 'Friday']
data = [['Springfield'], ['Shelbyville']]

for day in days:
    data[0].append(number_tracks(day, data[0][0]))
    data[1].append(number_tracks(day, data[1][0]))

table_h1 = pd.DataFrame(data = data, columns = columns)

In [52]:
table_h1

,city,monday,wednesday,friday
0,Springfield,15740,11056,15945
1,Shelbyville,5614,7003,5895


**Conclusões**

Podemos dizer que a hipótese um está correta, uma vez que a atividade do usuário variou de acordo com a cidade e dia da semana. Podemos observar que Shelbyville tem muito menos atividade do que a atividade de Springfield, com a diferença entre o número de músicas escutadas chegando a 10.126 em segundas-feira. Contudo, na quarta-feira a atividade em Shelbyville aumenta e essa diferença entre músicas escutadas cai para apenas 7003.

[Voltar ao Índice](#back)

### Hipótese 2: música no início e no final da semana <a id='week'></a>

De acordo com a segunda hipótese, na segunda-feira de manhã e na sexta-feira à noite, os cidadãos de Springfield ouvem gêneros diferentes dos que os utilizadores de Shelbyville gostam.

In [76]:
# criei a tabela spr_general a partir das linhas df
# onde o valor na coluna 'city' é 'Springfield'
spr_general = df[df['city'] == 'Springfield']
spr_general.reset_index(inplace=True)

In [77]:
spr_general

,index,user_id,track,artist,genre,city,time,day
0,1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
1,4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
2,6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
3,7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
4,8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
...,...,...,...,...,...,...,...,...
42736,65073,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Springfield,21:07:12,Monday
42737,65074,729CBB09,My Name,McLean,rnb,Springfield,13:32:28,Wednesday
42738,65076,C5E3A0D5,Jalopiina,unknown,industrial,Springfield,20:09:26,Friday
42739,65077,321D0506,Freight Train,Chas McDevitt,rock,Springfield,21:43:59,Friday


In [78]:
# criei o shel_general a partir das linhas df
# onde o valor na coluna 'city' é 'Shelbyville'
shel_general = df[df['city'] == 'Shelbyville']
shel_general.reset_index(inplace=True)

Escrevi a função `genre_weekday()` com quatro parâmetros:
* Uma tabela para dados (`df`)
* O dia da semana (`day`)
* O primeiro timestamp, no formato 'hh:mm:ss' (`time1`)
* O último timestamp, no formato 'hh:mm:ss' (`time2`)

A função deve retornar os 15 gêneros mais populares em um dia específico dentro do período definido pelos dois timestamps, juntamente com suas respectivas contagens de reprodução.
Apliquei a mesma lógica de filtragem consecutiva, mas use quatro filtros desta vez e, em seguida, fiz uma nova coluna com as respectivas contagens de reprodução.
Ordenei o resultado da contagem do maior para o menor e retornei-os.

In [72]:
# 1) Deixei a variável genre_df armazenar as linhas que satisfazem várias condições:
#    - o valor na coluna 'day' é igual ao valor do argumento day=
#    - o valor na coluna 'time' é maior do que o valor do argumento time1=
#    - o valor na coluna 'time' é menor do que o valor do argumento time2=
#    Usei filtros consecutivos com indexação lógica.

# 2) Agrupei genre_df pela coluna 'genre', peguei uma das suas colunas,
#    e usei o método size () para encontrar o número de entradas para cada um dos
#    representantes de gênero; armazenei o objeto Series resultante na
#    variável genre_df_count

# 3) Organizei genre_df_count em ordem decrescente de frequência e armazenei o resultado
#    para a variável genre_df_sorted

# 4) Retornei um objeto Series com os primeiros 15 valores genre_df_sorted - os 15 maiores
#    generos populares (em um determinado dia, dentro de um certo intervalo de tempo)

def genre_weekday(df, day, time1, time2):
    
    # Filtragem consecutiva
    # Criei a variável genre_df que armazenar apenas as linhas df onde o dia é igual a day=
    genre_df = df[df['day'] == day]
    # Filtrei genre_df novamente para armazenar apenas as linhas onde o tempo é menor que time2=
    genre_df = genre_df[df['time'] > time1]
    # Filtrei genre_df mais uma vez para armazenar apenas as linhas onde o tempo é maior que time1=
    genre_df = genre_df[df['time'] < time2]    

    # Agrupei o quadro de dados filtrado pela coluna com os nomes dos gêneros, selecionei a coluna ' gênero,
    # e encontrei o número de linhas para cada gênero com o método count()
    genre_df_count = genre_df.groupby('genre')['track'].count()
    
    # Ordenei o resultado em ordem decrescente (para que os gêneros mais populares venham primeiro no objeto Series)
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    
    # Retornei um objeto de série com os primeiros 15 valores de genre_df_sorted - os 15 gêneros mais populares (em um determinado dia, dentro de um determinado período de tempo)
    return genre_df_sorted[:15]

Comparei os resultados da função 'genre_weekday ()' para Springfield e Shelbyville na segunda-feira de manhã (das 07:00 às 11:00) e na sexta-feira à noite (das 17:00 às 23:00). Utilizei o mesmo formato de hora de 24 horas que o conjunto de dados (p. ex. 05:00 = 17:00:00):

In [111]:
# chamando a função para segunda-feira de manha em Springfield (use spr_general em vez de df table)
spr_genre_monday = genre_weekday(spr_general, 'Monday', '07:00:00', '11:00:00')
spr_genre_monday

/tmp/ipykernel_27/1092151784.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] > time1]
/tmp/ipykernel_27/1092151784.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] < time2]


genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: track, dtype: int64

In [112]:
# chamando a função para segunda-feira de manhã em Shelbyville (use shel_general em vez de df table)
shel_genre_monday = genre_weekday(shel_general, 'Monday', '07:00:00', '11:00:00')
shel_genre_monday

/tmp/ipykernel_27/1092151784.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] > time1]
/tmp/ipykernel_27/1092151784.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] < time2]


genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: track, dtype: int64

In [114]:
# chamando a função para sexta-feira à tarde em Springfield
spr_genre_friday = genre_weekday(spr_general, 'Friday', '17:00:00', '23:00:00')
spr_genre_friday

/tmp/ipykernel_27/1092151784.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] > time1]
/tmp/ipykernel_27/1092151784.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] < time2]


genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: track, dtype: int64

In [113]:
# chamando a função para sexta-feira à tarde em Shelbyville
shel_genre_friday = genre_weekday(shel_general, 'Friday', '17:00:00', '23:00:00')
shel_genre_friday

/tmp/ipykernel_27/1092151784.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] > time1]
/tmp/ipykernel_27/1092151784.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_df = genre_df[df['time'] < time2]


genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: track, dtype: int64

**Conclusão**

A segunda hipótese deve ser rejeitada, uma vez que é possível inspecionar com os resultados que as 15 categorias ouvidas em Shelbyville muito se assemelham as ouvidas em Springfield nas segundas-feiras de manhã e sexta-feira à noite.

In [115]:
#FUNÇÃO AUXILIAR: Verifica o número de categorias diferentes entre as 15 mais ouvidas em Shelbyville e as 15 mais ouvidas
# em Springfield
def num_categorias_diferentes(spr_genre, shel_genre):
    i = 0
    for key in spr_genre.keys():
        if key not in shel_genre.keys():
            i += 1
    return i

In [116]:
num_categorias_diferentes(s1,s2)

2

In [117]:
num_categorias_diferentes(s3,s4)

1

[Voltar ao Índice](#back)

### Hipótese 3: preferências de gênero em Springfield e Shelbyville <a id='genre'></a>

Hipótese: Shelbyville ama rap. Cidadãos de Springfield curtem mais pop.

Agrupei a tabela `spr_general` por gênero e encontrei o número de músicas tocadas para cada gênero usando o método `count()`. Então organizei o resultado em ordem decrescente e armazene-o na variável `spr_genres`.

In [119]:
spr_genres = spr_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Imprimi as primeiras 10 linhas de `spr_genres`:

In [121]:
# imprimindo as primeiras 10 linhas de spr_genres
spr_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Agora fiz o mesmo com os dados de Shelbyville.

Agrupei a tabela `shel_general` por gênero e encontrei o número de músicas tocadas para cada gênero. Depois organizei o resultado em ordem decrescente e armazene-o na tabela `shel_genres`:


In [ ]:
shel_genres = shel_general.groupby('genre')['genre'].count().sort_values(ascending=False)

exibindo as primeiras 10 linhas de `shel_genres`:

In [123]:
# imprimindo as primeiras 10 linhas de spr_genres
shel_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusão**


Parcialmente Rejeitada. os dados em "spr_genres" e "shel_genres" nos permite inferir que Springfield realmente ama pop, mas essa também é a preferência de Shelbyville. Ambas as cidades possuem preferência pelos gêneros "pop", "dance", "rock" e "eletronic".

[Voltar ao Índice](#back)

# Conclusões <a id='end'></a>


1ª Hipótese: Aceita. A atividade do usuário variou de acordo com a cidade e dia da semana;  
2ª Hipótese: Rejeitada. as 15 categorias ouvidas em Shelbyville são praticamente as mesmas ouvidas em Springfield nas segundas-feira de manhã e sextas-feira à noite.  
3ª Hipótese: Parcialmente Rejeitada. Shelbyville prefere o gênero pop, dance e rock ao invés do rap. Já Springfiel realmente ama pop.

### Importante
Esse é um projeto inicial. Em projetos de pesquisas reais, o teste estatístico de hipóteses é mais preciso e quantitativo. Observa-se também que conclusões sobre uma cidade inteira nem sempre podem ser tiradas a partir de dados de apenas uma fonte.

[Voltar ao Índice](#back)